# Student Progress (Anonymised) by Calculating Academic Vectors

 This script processes student academic data to create academic vectors for each student. We use this to determine
 - a student's suitability for higher level subjects
 - subject selection for Year 10-12
 - measuring the level of academic progress for students
 - metrics to facilitate conversations abour student academic progress

In [380]:
### IMPORT PACKAGES ###
#######################
import datetime
import pandas as pd
import pyodbc
import numpy as np
import matplotlib
import csv
import re
from pandas import DataFrame
%matplotlib inline

## Step 0. Connector


In [ ]:
connection = pyodbc.connect('Driver={SQL Server};'
    'Server=server_name;' # server_name anonymised
    'Database=db;'   # datanase anonymised
    'Trusted_Connection=yes;') 

## Step 1: Import from Database


In [381]:
def import_sql(conn):                                #accepts db connection as argument 
            
    cursor = conn.cursor()                           #Selecting ALL grades for ALL current/ past students
    cursor.execute("SELECT SA.[FileType]\
    ,SA.[FileYear]\
    ,SA.[FileSemester]\
    ,SA.[ID]\
    ,SM.YearLevel\
    ,SA.[ClassCampus]\
    ,SA.[ClassCode]\
    ,SA.[Result]\
    FROM [db].[dbo].[StudentAssessmentResults] SA\
    join pvStudentNamesall SM\
    on SM.FileYear = SA.FileYear\
    and SM.id = SA.ID\
    where SA.result in ('A','B','C','D','E')\
    and SA.filetype= 'A'\
    and not SA.result =''\
    and not SA.FileSemester = '1' UNION ALL SELECT  SB.[FileType],\
    SB.[FileYear]\
    ,SB.[FileSemester]\
    ,SB.[ID]\
    ,SN.YearLevel\
    ,SB.[ClassCampus]\
    ,SB.[ClassCode]\
    ,SB.[Result]\
    FROM [db].[dbo].[pastStudentAssessmentResults] SB\
    join pvStudentNamesall SN\
    on SN.FileYear = SB.FileYear\
    and SN.id = SB.ID\
    where  SB.result in ('A','B','C','D','E')\
    and SB.filetype= 'A'\
    and not SB.result =''\
    and not SB.FileSemester = 1")
    student_raw=cursor.fetchall()
    return student_raw

In [382]:
def import_classes(conn): #accepts db connection as argument 
    
    cursor = conn.cursor()    
    cursor.execute("SELECT DISTINCT ClassCode, LearningAreaCode\
    FROM SubjectClasses\
    WHERE FileType = 'A' AND FileYear >= 2011 AND AssessmentCode <> ''\
    ORDER BY ClassCode")
    list_raw=cursor.fetchall()
    return list_raw


In [383]:
def classlist_to_df():                   #converts class list to dataframe
    classy_list = import_classes()
    a=[0]*len(classy_list)
    b=[0]*len(classy_list)
    c=[0]*len(classy_list)
    for i in range(len(classy_list)):
        a[i] = classy_list[i][1]
        b[i] = classy_list[i][0]
        c[i] = i
    c = np.array(c)
    return pd.DataFrame({'L Area':a, 'ClassCode':b}, index =c)


In [384]:
def yes_to_one(t): # converting yes to one
    if bool(t['L Area'])==True:
        return 1
    else:
        return 0

In [385]:
def y_to_1(frame): # applying to dataframe
    frame['L Area2']=frame.apply(lambda x: yes_to_one(x), axis =1)
    return frame

In [386]:
def real_classlist(): #final and actual class lists 
    clt= classlist_to_df()
    clt2 = y_to_1(clt)
    clt3 = clt2.groupby('ClassCode').max()
    clt3.index.names = ['Class']
    clt3['ClassCode'] = clt3.index
    return clt3


### Excursus - 5.3/5.2/5.1 Mathematics Scaler

Scales 5.2 and 5.1 results so that the maximum a student can get is a C (only for calculating progress)

In [387]:
def math_query():  #query for maths classes
    return "select distinct ClassCode from subjectclasses where fileyear > 2008 and AssessmentCode like '%MAT%'and Description in ('Mathematics 5.1', 'Mathematics 5.1/2','Mathematics 5.1/5.2','Mathematics 5.2','Mathematics 5.2/5.1','9MATD','9MATE','9MATG','9MATF')"

In [ ]:
 def sql_convert(query_results, conn):
    return pd.read_sql_query(str(query_results),conn)

In [389]:
def math_scaler(row):
    if row['Result'] in ['A','B']:
        return 'C'
    else:
        return row['Result']


In [390]:
def math_52(frame):
    frame['Result']=frame.apply(lambda x: math_scaler(x), axis = 1)
    return frame

## Step 2.  Edit into DataFrame

In [391]:
def get_index(student_raw):                         #gets index for DataFrame
    student_raw_index = [0]*len(student_raw)
    for i in range(len(student_raw)):
        student_raw_index[i]= student_raw[i][3]
    return student_raw_index

In [392]:
def student_raw_series(student_raw):               #convert cursor to series
    student_raw_series = [0]*len(student_raw)
    for i in range(0,len(student_raw)):
        student_details = [0]*8 # 8 is the number of learning Areas at school
        for j in range (0,8):
            student_details[j]=student_raw[i][j]
        student_raw_series[i]=student_details
    return student_raw_series

In [393]:
def to_dataframe(series, student_raw_index):        #convert to DataFrame
    df=pd.DataFrame(series, index = student_raw_index, columns = ['FileType','FileYear','FileSemester','ID','YearLevel','ClassCampus','ClassCode','Result'])
    df=df.drop(['FileType','ClassCampus'], axis=1)
    df['FileYear']=df['FileYear'].astype(int)
    return df

In [394]:
def to_dataframe_for_compare(series, student_raw_index):     #removing columns
    df=pd.DataFrame(series, index = student_raw_index, columns = ['FileType','FileYear','FileSemester','ID','YearLevel','ClassCampus','ClassCode','Result'])
    df['FileYear']=df['FileYear'].astype(int)
    return df

In [395]:
#############################################
### PIPELINE FROM DATABASE TO DATAFRAME ###
#############################################

def data_pipeline():                              #pipeline from Database to DataFrame
    x = import_sql()
    y = get_index(x)
    z = student_raw_series(x)
    z = np.array(z)
    df = to_dataframe(z,y)
    return df

In [396]:
def year_before_now(row, minimum):
    return int(row['FileYear'])-minimum

In [397]:
def maths_only(frame):
    return frame[frame['ClassCode'].isin(list(sql_convert(math_query()).loc[:,'ClassCode']))]

In [398]:
def math_data_pipeline():     #transforms Mathematics results, to place on a common grade scale
    x = data_pipeline()
    x=x.reset_index()
    y=math_52(maths_only(x))
    x.update(y)
    x=x.set_index('ID')
    x=x.drop(columns='index')
    x['ID']=x.index
    x = x[['FileYear','FileSemester','ID','YearLevel','ClassCode','Result']]
    x = x.astype({"FileYear":int})
    return x



## Step 3. Replace Grades with Numerical Value

In [399]:
def grade_number(row):
    if row['Result'] in ['A', 'Exceeds expectations','Exceptional','Ex']:
        return 5
    elif row['Result'] in ['B','Competent','Commendable','Th','Co']:
        return 4
    elif row['Result'] in ['D', 'Ba','Below Expectations','Not yet competent','Below expectations']:
        return 2
    elif row['Result'] in ['E', 'Achieved with Support','Unsatisfactory','Unsatisfactory - needs improve']:
        return 1
    else:
        return 3

In [400]:
def drop_columns1(frame):
    frame.drop(frame.columns[[0,5]], axis=1, inplace=True)
    return frame

In [401]:
def result_number(frame):                          # calculate grade number
    frame['Result Number'] =frame.apply(lambda x: grade_number(x), axis =1)
    return frame

In [402]:
def class_subj(row):                               # extract class code using regex
    x = re.match(r"([0-9]+)([A-Z]+)([0-9-]+)([A-Z]+)", row['ClassCode'])
    if bool(re.match(r"([0-9]+)([A-Z]+)([0-9-]+)([A-Z]+)", row['ClassCode']))==True:
        return x.group(2)
    else:
        return 0

In [403]:
def insert_sub():
    x=import_classes()
    frame=classlist_to_df(x)
    frame['Subject']=frame.apply(lambda x: class_subj(x), axis =1)
    return frame

In [404]:
def add_subject(frame):
    frame['Subject']=frame.apply(lambda x: class_subj(x), axis =1)
    return frame

In [340]:
def add_years(frame,minimum):
    frame['Years Ago']=frame.apply(lambda x: year_before_now(x, minimum)+1, axis=1)
    return frame

## Step 4. Group into Learning Areas

In [341]:
def _filter(frame):
    frame['Subject'] = frame.apply(lambda x:subj_convert(x), axis=1)                         
    return frame[frame['Subject'].isin(['ENG','MAT','SCI','PDH','MUS', 'GEO','IND','MUS'])]  # all learning areas except languages

In [342]:
def add_weighted_mark(frame):
    frame['Weighted Mark']=frame['Years Ago']*frame['Result Number']     #a  weighted average, the more recent ones worth more
    return frame

In [343]:
def filter_only(frame):
    return frame[['FileYear','FileSemester','ID','YearLevel','L Area', 'Years Ago','Weighted Mark']]

In [344]:
def group_by_sum(frame):                               
    return frame.groupby(['ID', 'L Area']).sum()

In [345]:
def add_weighted_av(frame):
    frame['Weighted Average']=frame['Weighted Mark']/frame['Years Ago'] 
    return frame

## Step 5: Derive Academic Vectors

In [346]:
#######################################
###   ACADEMIC VECTORS - VECTORS    ### 
### CONTAINING STUDENT ABILITIES    ###
###         AT TIME 'year'          ###
#######################################b

def academic_vector(year):
    frame = math_data_pipeline()
    frame['ID']=frame['ID'].astype(int)
    res = result_number(frame)
    adds = add_subject(res)
    subj=real_classlist()
    filt1 = adds.merge(subj,on='ClassCode').drop(columns=['Subject', 'L Area2'])
    filt1 = filt1[filt1['FileYear']<=year]
    minimum = min(adds['FileYear'])
    addy = add_years(filt1,minimum)
    wm = add_weighted_mark(addy)
    filt2=filter_only(wm)
    group = group_by_sum(filt2)
    wa = add_weighted_av(group)
    WA=wa.reset_index()
    WA_pivot = WA.pivot(index='ID', columns='L Area', values='Weighted Average')
    WA_pivot = WA_pivot.fillna(3)   # new grades at 3
    WA_pivot['Average']=(WA_pivot['ENG']+WA_pivot['MAT']+WA_pivot['SCI']+WA_pivot['PDH'])
    return WA_pivot

In [1]:
# Convert subject names to Learning Areas

def subj_convert(row):
    if row['Subject'] in ['BHI','BRL','CHD','CHDX','DAN','FLM','FOT','FRE','FREB','FREC','GER',
               'GRA','INE','JAP','JAPB','MHI', 'PAS','PDM','PRG','RSS','SNT','SPS']:
        return 'Average'    
    
    elif row['Subject'] in ['DRW', 'VIS','TEC','SDD','IND','TEX','IPT','IST','MCC']:
        return 'IND'
    
    elif row['Subject'] in ['BUS', 'COM', 'AHI', 'HIS', 'LEG','SOC', 'SOR','SCH', 'HSI','ECO','GEQ','GEO', 'HMI', 'SOR']:
        return 'GEO'
    
    elif row['Subject'] in ['EES','EGS','CHE','BIO','PHY','SCI']:
        return 'SCI'
    
    elif row['Subject'] in ['DRA', 'MUS', 'MUT']:
        return 'MUS'
        
    elif row['Subject'] in ['ENG','ENGA','ENGS','ENGX']:
        return 'ENG'
    
    elif row['Subject'] in ['PDH','PAS', 'DAN']:
        return 'PDH'
    
    elif row['Subject'] in ['MAT','MATG','MATX']:
        return 'MAT'

    else:
        return 'Average'


In [350]:
def import_sql_compare(year, semester):
    while True:
        try:
            if year <2017:
                print('Year must be 2017 or greater')
            else:
                year=int(year)
        except:
            print('This must be an integer greater than 2017')
            break
        else: 
            if semester not in [2,4]:
                print("this is not semester 2 or 4")
                break
            else:
                while True:
                    try:
                        semester=int(semester)
                    except:
                        print('This must be an integer')
                        break
                    else:
                        conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=server_name;' # server_name anonymised
                        'Database=db;'   # datanase anonymised
                        'Trusted_Connection=yes;')     

                        cursor = conn.cursor()
                        cursor.execute("SELECT  sa.[FileType]\
                        ,SA.[FileYear]\
                        ,sa.[FileSemester] \
                        ,SA.[ID]\
                        ,SN.YearLevel\
                        ,sa.[ClassCampus] \
                        ,sa.[ClassCode]\
                        ,sa.[Result]\
                        FROM [db].[dbo].[StudentAssessmentResults] SA\
                        join pvStudentNames SN\
                        on SN.FileYear = SA.FileYear\
                        and SN.id = SA.ID\
                        where  sa.result in ('A','B','C','D','E')\
                        and sa.filetype= 'A'\
                        and not sa.result =''\
                        and SA.fileyear =" + str(year)+"\
                        and sa.FileSemester =" + str(semester)+"\
                        UNION ALL SELECT distinct SB.[FileType],\
                        SB.[FileYear]\
                        ,SB.[FileSemester]\
                        ,SB.[ID]\
                        ,SM.YearLevel\
                        ,SB.[ClassCampus]\
                        ,SB.[ClassCode]\
                        ,SB.[Result]\
                        FROM [db].[dbo].[pastStudentAssessmentResults] SB\
                        join pvStudentNamesall SM\
                        on SM.FileYear = SB.FileYear\
                        and SM.id = SB.ID\
                        where  SB.result in ('A','B','C','D','E')\
                        and SB.filetype= 'A'\
                        and not SB.result =''\
                        and SB.fileyear =" + str(year)+"\
                        and SB.FileSemester =" + str(semester))
                        student_raw=cursor.fetchall()
                        y = get_index(student_raw)
                        z = student_raw_series(student_raw)
                        z = np.array(z)
                        df = to_dataframe_for_compare(z,y)
                        df.reset_index(inplace=True)
                        df=df.drop(['index'], axis=1)
                        return df

## Step 6. Some HSC Methods

In [351]:
def subject_name(row):                               # extract class code using regex
    x = re.match(r"([0-9]+)( unit )([A-Za-z12 \' ]+)", row['Course'])
    if bool(x)==True:
        return x.group(3)
    else:
        return 0

def insert_subject(frame):
    frame['Course Title'] = frame.apply(lambda x: subject_name(x), axis=1)
    return frame


In [352]:
def stu_nesa(conn):    #accepts conn as Connector
    cursor = conn.cursor()                           #Selecting ALL grades for ALL current/ past students
    cursor.execute("SELECT [ID]\
      ,[Surname]\
      ,[Preferred]\
      ,[StudiesCode]\
      FROM [db].[dbo].[vPastStudentAddress]")

    stu_numbers=cursor.fetchall()
    a=[0]*len(stu_numbers)
    b=[0]*len(stu_numbers)
    c=[0]*len(stu_numbers)
    d=[0]*len(stu_numbers)
    e=[0]*len(stu_numbers)
    for i in range(len(stu_numbers)):
        a[i] = stu_numbers[i][0]
        b[i] = stu_numbers[i][1]
        c[i] = stu_numbers[i][2]
        d[i] = stu_numbers[i][3]
        e[i] = i
    e = np.array(e)
    return pd.DataFrame({'ID':a, 'Surname':b, 'Preferred':c, 'Student Id':d}, index =e)

In [353]:
def hsc_performance():
    x = [0]*10
    for i in range(2009, 2019):
         x[i-2009]= pd.read_csv("\home\All HSC results" + "\\" +str(i) +".csv" )
         x[i-2009]['Year'] = i
        
    for i in range(1,10):
        x[i]= pd.concat([x[i],x[i-1]], ignore_index = True)  
    
    allhsc = x[9]
    allhsc['Student Id'] = allhsc['Student Id'].astype(str)
    allhsc = insert_subject(allhsc)
    allhsc = allhsc.drop(allhsc.columns[[4,5,12,13,15,16,14]],axis=1, inplace=False)
    nesa_data = stu_nesa()
    df = nesa_data.merge(allhsc, on ='Student Id')
    df = df.drop(df.columns[[4,5,6]], axis=1, inplace = False)
    return DataFrame(df)

In [354]:
def hsc_merge():
    hsc = hsc_performance()
    hsc_new = DataFrame(hsc)
    hsc_new = hsc_new.rename(columns={'Surname_x':'Surname', 'Student Id':'NESA ID'})
    return hsc_new

## Step 6: Compare Report Grades to Academic Vectors to determine growth

Here, we use Rasch assessment design to calculate progress probabilities. The probability is based on the logit function to determine whether students who have shown a particular ability have scored beyond their expected performance level. 

We select an appropriate value k which scales results appropriate for our context. Here, k = 0.15

The form of the Rasch model is

 $$P(achievement>=x) = \frac{e^{a+x(k-1)}}{1+e^{a+x(k-1)}}$$
 
 Here, $P$ represents Progress Probability

Please see https://en.wikipedia.org/wiki/Rasch_model

In [355]:
###   ACADEMIC PROGRESS - AN AVERAGE    ### 
###  'PROBABILITY' OF STUDENT MOVEMENT  ###
###   AT SELECTED REPORTING  PERIOD     ###
###########################################

def academic_progress_bysubject(year, term):
    ability = academic_vector(year-1)
    selected_results = import_sql_compare(year,term)
    imp_cl = import_classes()
    selected_results['ID']=selected_results['ID'].astype(int)
    selected_results['YearLevel']=selected_results['YearLevel'].astype(int)
    col1 = drop_columns1(selected_results)
    res = result_number(col1)
    adds = add_subject(res)
    subj= real_classlist()
    filt = adds.merge(subj, on="ClassCode")
    performance = filt.groupby(['ID','L Area']).mean()
    performance.reset_index(inplace=True)
    tuner = 0.15
    
    k=[0]*len(performance)
    for i in range(len(performance)):
        if performance.loc[i]['ID'] in list(ability.index):
            k[i]=ability.loc[performance.loc[i]['ID'],performance.loc[i]['L Area']]
        else:
            k[i]=performance[['Result Number']].iloc[i,0]
            
    performance['Ability']=k
    performance['Probability']= np.exp( performance['Ability']-performance['Result Number']+tuner*performance['Result Number'])/( 1+np.exp( performance['Ability']-performance['Result Number']+tuner*performance['Result Number']))
    return performance

In [359]:
progress_results= academic_progress_bysubject(2020,2)

C:\Users\maa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
